# 类别数据（Categorical）

使用 Pandas 能够处理从几行到几百万行的数据集。在 Pandas 底层，使用 Numpy 的数组对象来存储数据集。Pandas支持多种数据类型，例如数值型、字符串、日期等。整数与浮点数还具有多个子类型，以便使用较少字节表示不同数据，例如浮点型就有 float16、float32 和 float64 等子类型，下表列出 Pandas 常用数据类型及其占用内存字节：

|内存      | float  |  int | uint | datetime | bool | object |
|:----------:|:------:|:----:|:------:|:------:|:-----:|:-----:|
|1 bytes    |      | int8 | uint8  |      | bool  |     |
|2 bytes    | float16| int16| uint16 |      |     |     |  
|4 bytes    | float32| int32| uint32 |      |     |     | 
|8 bytes    | float64| int64| uint64 | datetime64 |  |     |
|variable   |      |    |      |        |    | object |

本节介绍 Pandas 的特殊数据类型：类别数据（Categorical）。

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 类别数据（Categorical）的由来

前面章节提过，根据数据的类型与意义可分为：数值数据（Numerical data）；分类数据（Categorical data ）；有序数据（Ordinal data）。对于分类数据可以使用`value_counts()`方法提取不同值并计算频次。例如，下面给出用餐时间与用餐标识：

In [54]:
# 时间, 用餐标示
times = [6, 7, 8, 11, 12, 13, 18, 19, 20, 21] * 1024
meals = ['breakfast', 'breakfast', 'breakfast', 'lunch', 'lunch', 'lunch', 'dinner', 'dinner', 'dinner', 'dinner'] * 1024
ser = pd.Series(meals, index=times)

In [55]:
# 数据类型
print(ser.dtypes)
# 内存
print(len(ser), ser.memory_usage(deep=True))

object
10240 888832


In [56]:
# 唯一值与频次
ser.value_counts()

dinner       4096
breakfast    3072
lunch        3072
dtype: int64

由上可知，`Series`对象有 10240 个数据，唯一值只有3个，分别对应早餐、午餐与晚餐。数据类型为字符串对象(object)，占用内存大约为1233字节。

在 Numpy 与Pandas 中使用 object 类型来表示 Python 字符串对象的值，即 object 列中的每个元素实际存放的是字符串真实位置的指针，这导致字符串是以一种碎片化方式进行存储，内存占用多且访问速度慢。

Pandas 自 0.15 版开始引入类别数据类型，在底层使用整型数值而非原值来表示类别数据，再使用字典构建整型数据与原数据的映射关系。对于分类数据，使用类别会大大节省内存占用，且提高性能。下图展示一个包含个颜色的类别数据的示意图：
![类别（categoricalas）类型](../images/categorical.svg)

使用`Series`对象的方法`astype()`来转换字符串类型为类别数据：

In [71]:
ser2 = ser.astype('category')
ser2.head()

pandas.core.series.Series

6     breakfast
7     breakfast
8     breakfast
11        lunch
12        lunch
dtype: category
Categories (3, object): [breakfast, dinner, lunch]

再来看看底层发生了什么，使用`Series.cat.codes`属性来返回类别数据用以表示每个值的整型数字：

In [68]:
ser2.head().cat.codes

6     0
7     0
8     0
11    2
12    2
dtype: int8

可以看到，`Series`对象的每个数据都被赋值为一个整数，且为 int8 类型，分别用`0,1,2`标示。如果有缺失数据，会设为`-1`。

最后，再来看看转换为类别数据前后的内存使用量。

In [69]:
print(ser.memory_usage(deep=True))
print(ser2.memory_usage(deep=True))

888832
92477


内存占用率从 888 K 降到92 K，近乎90%的降幅！注意：仅为特例，包含近 10240 个数据的列只有 3 个唯一值。

## 类别数据的 Dummy 处理

在使用机器学习进行数据分析过程中，通常会将分类数据转换为独热（one-hot）编码或哑变量（Dummy)。Dummy 处理结果会返回一个 `DataFrame` 对象，每个唯一类别都是它的一列，用1表示出现，用0表示未出现。

Pandas 提供`pd.get_dummies()`函数将一维分类数据转换为独热编码数据帧：

In [73]:
df = pd.get_dummies(ser2)
df.head()

,breakfast,dinner,lunch
6,1,0,0
7,1,0,0
8,1,0,0
11,0,0,1
12,0,0,1
